# Process

KFold
  - DeepFM

  - AutoML

  - Trasnformer

In [ ]:
!pip install mljar-supervised

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.9/126.9 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pandas-2.1.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.3 MB)
  Using cached catboost-1.2.2-cp310-cp310-manylinux2014_x86_64.whl (98.7 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.5/533.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 21.6 MB/s eta 0:00:00
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.0 MB/s eta 0:00:00
  Created wheel for mljar-supervised: filename=mljar_supervised-1.1.1-py3-n

In [ ]:
#!pip install --upgrade autogluon

In [ ]:
import pandas as pd
import numpy as np
import random
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from category_encoders.target_encoder import TargetEncoder
import warnings
from supervised import AutoML
from google.colab import drive
import datetime as dt
import zipfile
#from autogluon.tabular import TabularDataset, TabularPredictor
#from autogluon.core.metrics import make_scorer

warnings.filterwarnings('ignore')

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive_maindir = '/content/drive/MyDrive/'
#os.mkdir(drive_maindir+'danger')
drive_zipmaindir = '/content/drive/MyDrive/open.zip'

In [ ]:
# zipfile = zipfile.ZipFile(drive_zipmaindir)
# zipfile.extractall(path = drive_maindir+'danger')

In [ ]:
drive_dangerdir = drive_maindir+'danger/open/'
train = pd.read_csv(drive_dangerdir + 'train.csv')
test = pd.read_csv(drive_dangerdir + 'test.csv')
sample_submission = pd.read_csv(drive_dangerdir + 'sample_submission.csv')

In [ ]:
# 도로유형 사고 위험도 관계
# ex) 국도, 지방도, 고속도로 등

In [ ]:
train = train[['ID', '사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', 'ECLO']]

In [ ]:
train['사고일시'] = pd.to_datetime(train['사고일시'])
test['사고일시'] = pd.to_datetime(test['사고일시'])

In [ ]:
train['도시'] = train['시군구'].str.split(' ').str[0]
train['구'] = train['시군구'].str.split(' ').str[1]
train['동'] = train['시군구'].str.split(' ').str[2]
test['도시'] = test['시군구'].str.split(' ').str[0]
test['구'] = test['시군구'].str.split(' ').str[1]
test['동'] = test['시군구'].str.split(' ').str[2]

train['사고일시'] = pd.to_datetime(train['사고일시'])
train['사고연도'] = train['사고일시'].dt.year.astype(int)
train['사고월'] = train['사고일시'].dt.month.astype(int)
train['사고일'] = train['사고일시'].dt.day.astype(int)
train['사고요일'] = train['사고일시'].dt.dayofweek.astype(int)
train['사고시간'] = train['사고일시'].dt.hour.astype(int)
train.drop(['사고일시'], axis = 1, inplace = True)
test['사고일시'] = pd.to_datetime(test['사고일시'])
test['사고연도'] = test['사고일시'].dt.year.astype(int)
test['사고월'] = test['사고일시'].dt.month.astype(int)
test['사고일'] = test['사고일시'].dt.day.astype(int)
test['사고요일'] = test['사고일시'].dt.dayofweek.astype(int)
test['사고시간'] = test['사고일시'].dt.hour.astype(int)
test.drop(['사고일시'], axis = 1, inplace = True)

In [ ]:
def get1(df):
  df['sin_사고시간'] = np.sin(2*np.pi*df['사고시간']/24)
  df['cos_사고시간'] = np.cos(2*np.pi*df['사고시간']/24)
  return df

train = get1(train)
test = get1(test)

# 계절
def get2(df):
  df['계절'] = np.nan
  df.loc[(df['사고월'] == 3) | (df['사고월'] == 4) | (df['사고월'] == 5), '계절'] = '봄'
  df.loc[(df['사고월'] == 6) | (df['사고월'] == 7) | (df['사고월'] == 8), '계절'] = '여름'
  df.loc[(df['사고월'] == 9) | (df['사고월'] == 10) | (df['사고월'] == 11), '계절'] = '가을'
  df.loc[(df['사고월'] == 12) | (df['사고월'] == 1) | (df['사고월'] == 2), '계절'] = '겨울'
  return df['계절']

train['계절'] = get2(train)
test['계절'] = get2(test)

# 주말여부
def get3(df):
  df['사고주말여부'] = np.nan
  df.loc[(df['사고요일'] < 5), '사고주말여부'] = 0
  df.loc[(df['사고요일'] >= 5), '사고주말여부'] = 1
  return df['사고주말여부']

train['사고주말여부'] = get3(train)
test['사고주말여부'] = get3(test)

# 시간대
def get4(df):
  df['시간대'] = np.nan
  df.loc[(df['사고시간'] < 6), '시간대'] = '새벽'
  df.loc[(df['사고시간'] >=6) & (df['사고시간'] < 12), '시간대'] = '아침'
  df.loc[(df['사고시간'] >= 12) & (df['사고시간'] < 19), '시간대'] = '오후'
  df.loc[(df['사고시간'] >= 19) & (df['사고시간'] <= 24), '시간대'] = '저녁'
  return df['시간대']

train['시간대'] = get4(train)
test['시간대'] = get4(test)

# 공휴일

# 공휴일 전

# 공휴일 후

외부 데이터

In [ ]:
drive_dangerexternaldir = drive_dangerdir+'external_open/'

# 대구 빅데이터 마트 정보
#

external_data2 = pd.read_csv(drive_dangerexternaldir + 'countrywide_accident.csv') # equal = train
external_data3 = pd.read_csv(drive_dangerexternaldir + '대구 CCTV 정보.csv', encoding = 'cp949') # check
external_data4 = pd.read_csv(drive_dangerexternaldir + '대구 보안등 정보.csv', encoding = 'cp949') # check
external_data5 = pd.read_csv(drive_dangerexternaldir + '대구 어린이 보호 구역 정보.csv', encoding = 'cp949') # check
external_data6 = pd.read_csv(drive_dangerexternaldir + '대구 주차장 정보.csv', encoding = 'cp949') #check

In [ ]:
def process_location_pattern(df, address_column, drop_columns):
  location_pattern = r'(\S+) (\S+) (\S+) (\S+)'
  df[['도시', '구', '동', '번지']] = df[address_column].str.extract(location_pattern)
  df.drop(columns = drop_columns, axis = 1, inplace = True)
  return df

In [ ]:
# CCTV
# 도로노선방향, 위도, 경도, 단속구분, 제한속도, 설치연도
external_data3 = process_location_pattern(external_data3, '소재지지번주소', ['번지', '소재지지번주소'])
external_data3_grouped = external_data3.groupby(['도시', '구', '동'])[['도로노선방향', '위도', '경도', '단속구분', '제한속도', '설치연도']].mean().reset_index()
external_data3_grouped.rename(columns={'위도': 'CCTV_위도', '경도': 'CCTV_경도'}, inplace=True)
external_data3_grouped.reset_index(inplace = True, drop = True)

# (설치연도 - 사고일시)?

In [ ]:
external_data3_notnull = external_data3.dropna(subset = ['도시', '구', '동'])
cctv_count = external_data3_notnull.groupby(['도시', '구', '동']).size().reset_index(name = 'CCTV개수')

In [ ]:
# 보안등
# 위도, 경도, 설치연도, 설치형태, 설치개수

external_data4 = process_location_pattern(external_data4, '소재지지번주소', ['번지', '소재지지번주소'])
external_data4 = pd.get_dummies(external_data4, columns = ['설치형태'], drop_first = True)

external_data4_grouped1 = external_data4.groupby(['도시', '구', '동'])[['위도', '경도', '설치연도']].mean().reset_index()
external_data4_grouped1.rename(columns={'위도': '보안등_위도', '경도': '보안등_경도'}, inplace=True)

external_data4_grouped2 = external_data4.groupby(['도시', '구', '동'])[external_data4.columns[external_data4.columns.str.contains('설치형태')]].sum().reset_index()

external_data4_grouped3 = external_data4.groupby(['도시', '구', '동'])['설치개수'].sum().reset_index()

external_data4_grouped = pd.merge(external_data4_grouped1, external_data4_grouped2, on = ['도시', '구', '동'], how = 'left')
external_data4_grouped = pd.merge(external_data4_grouped, external_data4_grouped3, on = ['도시', '구', '동'], how = 'left')
external_data4_grouped.reset_index(inplace = True, drop = True)

In [ ]:
def calculate_average(value_str):
  if '~' in value_str:
    start, end = map(float, value_str.split('~'))
    return (start + end) / 2
  if '-' in value_str:
    start, end = map(float, value_str.split('-'))
    return (start + end) / 2
  else:
    return float(value_str)

external_data5['보호구역도로폭'] = external_data5['보호구역도로폭'].astype(str).apply(calculate_average)

In [ ]:
# 어린이 보호 구역
# 소재지지번주소, 위도, 경도, CCTV설치대수, 보호구역도로폭

external_data5 = process_location_pattern(external_data5, '소재지지번주소', ['번지', '소재지지번주소'])

external_data5_grouped1 = external_data5.groupby(['도시', '구', '동'])[['위도', '경도', '보호구역도로폭']].mean().reset_index()
external_data5_grouped1.rename(columns={'위도': '어린이보호구역_위도', '경도': '어린이보호구역_경도'}, inplace=True)

external_data5_grouped2 = external_data5.groupby(['도시', '구', '동'])['CCTV설치대수'].sum().reset_index()

external_data5_grouped = pd.merge(external_data5_grouped1, external_data5_grouped2, on = ['도시', '구', '동'], how = 'left')
external_data5_grouped.reset_index(inplace = True, drop = True)

In [ ]:
# 초등학교 개수
external_data5_n = external_data5.groupby(['도시', '구', '동'])['대상시설명'].nunique().reset_index()
external_data5_n.columns = ['도시', '구', '동', '초등학교개수']

In [ ]:
# 주차장
external_data6 = process_location_pattern(external_data6, '소재지지번주소', ['번지', '소재지지번주소'])

external_data6_grouped = external_data6.groupby(['도시', '구', '동'])[['위도', '경도']].mean().reset_index()
external_data6_grouped.rename(columns={'위도': '주차장_위도', '경도': '주차장_경도'}, inplace=True)


# 급지구분
external_data6_p = pd.get_dummies(external_data6[['도시', '구', '동', '급지구분']], columns = ['급지구분'], prefix = '급지')

# 주차장 개수
external_data6_n = external_data6_p.groupby(['도시', '구', '동']).size().reset_index()
external_data6_n.rename(columns = {0: '주차장개수'}, inplace = True)

# 주차장 유형 개수
external_data6_park = pd.get_dummies(external_data6[['도시', '구', '동', '주차장유형']], columns = ['주차장유형'], prefix = '주차장유형')
external_data6_park_n = external_data6_park.groupby(['도시', '구', '동'])[external_data6_park.columns[external_data6_park.columns.str.contains('주차장유형')]].sum().reset_index()

external_data6_sum = pd.merge(external_data6_n, external_data6_p, on = ['도시', '구', '동'], how = 'left')
external_data6_sum = external_data6_sum.groupby(['도시', '구', '동']).sum().reset_index()
external_data6_sum = pd.merge(external_data6_sum, external_data6_grouped, on = ['도시', '구', '동'], how = 'left')
external_data6_sum = pd.merge(external_data6_sum, external_data6_park_n, on = ['도시', '구', '동'], how = 'left')

In [ ]:
train = pd.merge(train, external_data3_grouped, on = ['도시', '구', '동'], how = 'left')
train = pd.merge(train, cctv_count, on = ['도시', '구', '동'], how = 'left')
train = pd.merge(train, external_data4_grouped, on = ['도시', '구', '동'], how = 'left')
train = pd.merge(train, external_data5_grouped, on = ['도시', '구', '동'], how = 'left')
train = pd.merge(train, external_data5_n, on = ['도시', '구', '동'], how = 'left')
train = pd.merge(train, external_data6_sum, on = ['도시', '구', '동'], how = 'left')

test = pd.merge(test, external_data3_grouped, on = ['도시', '구', '동'], how = 'left')
test = pd.merge(test, cctv_count, on = ['도시', '구', '동'], how = 'left')
test = pd.merge(test, external_data4_grouped, on = ['도시', '구', '동'], how = 'left')
test = pd.merge(test, external_data5_grouped, on = ['도시', '구', '동'], how = 'left')
test = pd.merge(test, external_data5_n, on = ['도시', '구', '동'], how = 'left')
test = pd.merge(test, external_data6_sum, on = ['도시', '구', '동'], how = 'left')

In [ ]:
# train = pd.get_dummies(data = train, columns = ['요일', '기상상태', '도로형태', '노면상태', '사고유형'])
train.drop(['도시'], axis = 1, inplace = True)
# test = pd.get_dummies(data = test, columns = ['요일', '기상상태', '도로형태', '노면상태', '사고유형'])
test.drop(['도시'], axis = 1, inplace = True)

In [ ]:
train_col = list(train.columns)
test_col = list(test.columns)
list(set(train_col) - set(test_col))

['ECLO']

In [ ]:
foggy = [0] * len(test)
test['기상상태_안개'] = foggy

In [ ]:
train.isnull().sum()[train.isnull().sum() > 0]

도로노선방향         1576
CCTV_위도        1576
CCTV_경도        1576
단속구분           1576
제한속도           1576
설치연도_x         1576
CCTV개수         1576
보안등_위도        13065
보안등_경도        13065
설치연도_y        23629
설치형태_전용주       9513
설치형태_한전주       9513
설치개수           9513
어린이보호구역_위도    18426
어린이보호구역_경도    18426
보호구역도로폭       26966
CCTV설치대수      18426
초등학교개수        18426
주차장개수          6543
급지_1           6543
급지_2           6543
급지_3           6543
주차장_위도        12231
주차장_경도        12231
주차장유형_노상       6543
주차장유형_노외       6543
dtype: int64

In [ ]:
test.isnull().sum()[test.isnull().sum() > 0]

도로노선방향         455
CCTV_위도        455
CCTV_경도        455
단속구분           455
제한속도           455
설치연도_x         455
CCTV개수         455
보안등_위도        3795
보안등_경도        3795
설치연도_y        6722
설치형태_전용주      2771
설치형태_한전주      2771
설치개수          2771
어린이보호구역_위도    4961
어린이보호구역_경도    4961
보호구역도로폭       7465
CCTV설치대수      4961
초등학교개수        4961
주차장개수         1928
급지_1          1928
급지_2          1928
급지_3          1928
주차장_위도        3510
주차장_경도        3510
주차장유형_노상      1928
주차장유형_노외      1928
dtype: int64

In [ ]:
#!pip install haversine

In [ ]:
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.decomposition import PCA, TruncatedSVD
#from haversine import haversine
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

In [ ]:
# # 위도, 경도 클러스터링
# # group0, group1 <-> group2
# # group1 <-> group3

group0 = ['CCTV_위도', 'CCTV_경도']
group1 = ['보안등_위도', '보안등_경도']
group2 = ['어린이보호구역_위도', '어린이보호구역_경도']
group3 = ['주차장_위도', '주차장_경도']

# def kmeans_best_k(df, group):
#   silhouette_scores = []
#   for k in range(1, 13):
#       kmeans = KMeans(n_clusters=k, random_state=42)
#       labels = kmeans.fit_predict(df[group])
#       silhouette_scores.append(silhouette_score(df[group], labels))

#   plt.plot(range(1, 13), silhouette_scores, marker='o')
#   plt.xlabel('Number of Clusters (k)')
#   plt.ylabel('Silhouette Score')
#   plt.title('Silhouette Score Method')
#   plt.show()

# kmeans_best_k(train, group0)
# kmeans_best_k(train, group1)
# kmeans_best_k(train, group2)
# kmeans_best_k(train, group3)

In [ ]:
# # 거리

# def get_dist(df, group_s, group_e):
#   start = tuple(zip(df[group_s[0]], df[group_s[1]]))
#   end = tuple(zip(df[group_e[0]], df[group_e[1]]))
#   dist = [haversine(s, e) for s, e in zip(start, end)]
#   return dist

# train['CCTV_어린이보호구역_거리'] = get_dist(train, group0, group2)
# train['보안등_어린이보호구역_거리'] = get_dist(train, group1, group2)
# train['보안등_주차장_거리'] = get_dist(train, group1, group3)
# train['주차장_어린이보호구역_거리'] = get_dist(train, group3, group2)

# test['CCTV_어린이보호구역_거리'] = get_dist(test, group0, group2)
# test['보안등_어린이보호구역_거리'] = get_dist(test, group1, group2)
# test['보안등_주차장_거리'] = get_dist(test, group1, group3)
# test['주차장_어린이보호구역_거리'] = get_dist(test, group3, group2)

# Modeling

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
def rmsle(y, pred):
  log_y = np.log1p(y)
  log_pred = np.log1p(pred)
  squared_error = (log_y-log_pred)**2
  rmsle = np.sqrt(np.mean(squared_error))
  return rmsle

def rmse(y, pred):
  return mean_squared_error(y, pred)**0.5

In [ ]:
def seed_everything(seed: int = 42):
  random.seed(seed)
  np.random.seed(seed)
  os.environ["PYTHONHASHSEED"] = str(seed)

In [ ]:
train.columns

Index(['ID', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', 'ECLO', '구', '동',
       '사고연도', '사고월', '사고일', '사고요일', '사고시간', 'sin_사고시간', 'cos_사고시간', '계절',
       '사고주말여부', '시간대', '도로노선방향', 'CCTV_위도', 'CCTV_경도', '단속구분', '제한속도',
       '설치연도_x', 'CCTV개수', '보안등_위도', '보안등_경도', '설치연도_y', '설치형태_전용주',
       '설치형태_한전주', '설치개수', '어린이보호구역_위도', '어린이보호구역_경도', '보호구역도로폭', 'CCTV설치대수',
       '초등학교개수', '주차장개수', '급지_1', '급지_2', '급지_3', '주차장_위도', '주차장_경도',
       '주차장유형_노상', '주차장유형_노외'],
      dtype='object')

In [ ]:
drops = train.columns[train.columns.str.contains('위도')]
train.drop(drops, axis = 1, inplace = True)
test.drop(drops, axis = 1, inplace = True)

In [ ]:
drops = train.columns[train.columns.str.contains('경도')]
train.drop(drops, axis = 1, inplace = True)
test.drop(drops, axis = 1, inplace = True)

In [ ]:
# drops = train.columns[train.columns.str.contains('요일_')]
# train.drop(drops, axis = 1, inplace = True)
# test.drop(drops, axis = 1, inplace = True)

In [ ]:
drops = ['ID', '사고연도', '사고월', '설치연도_x', '설치연도_y', '사고일', '사고시간']
train.drop(drops, axis = 1, inplace = True)
test.drop(drops, axis = 1, inplace = True)

In [ ]:
train.columns

Index(['요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', 'ECLO', '구', '동', '사고요일',
       'sin_사고시간', 'cos_사고시간', '계절', '사고주말여부', '시간대', '도로노선방향', '단속구분', '제한속도',
       'CCTV개수', '설치형태_전용주', '설치형태_한전주', '설치개수', '보호구역도로폭', 'CCTV설치대수',
       '초등학교개수', '주차장개수', '급지_1', '급지_2', '급지_3', '주차장유형_노상', '주차장유형_노외'],
      dtype='object')

In [ ]:
accident_counts = train['시군구'].value_counts().reset_index()
accident_counts.columns = ['시군구', '사고횟수']

train['사고발생횟수'] = train['시군구'].map(accident_counts.set_index('시군구')['사고횟수'])
test['사고발생횟수'] = test['시군구'].map(accident_counts.set_index('시군구')['사고횟수'])

In [ ]:
# 범주형 특성
sparse_features = ['요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', '구', '동', '사고요일', '계절', '사고주말여부', '시간대', '도로노선방향', '단속구분', '제한속도', '보호구역도로폭']

# 숫자형 특성
dense_features = ['CCTV개수', '설치개수', 'CCTV설치대수', '설치형태_전용주', '설치형태_한전주', '초등학교개수', '주차장개수', '급지_1', '급지_2', '급지_3', '주차장유형_노상', '주차장유형_노외', '사고발생횟수']

train[sparse_features] = train[sparse_features].fillna(-1)
train[dense_features] = train[dense_features].fillna(0)
test[sparse_features] = test[sparse_features].fillna(-1)
test[dense_features] = test[dense_features].fillna(0)

In [ ]:
for col in train.drop(['ECLO'], axis = 1).columns:
  if (train[col].dtype == 'object'):
    # tr_encoder = TargetEncoder(cols=[col])
    # train[col] = tr_encoder.fit_transform(train[col], train['ECLO'])
    # test[col] = tr_encoder.transform(test[col])
    encoder = LabelEncoder()
    train[col] = encoder.fit_transform(train[col])
    test[col] = encoder.transform(test[col])
  elif train[col].dtype == 'bool':
    encoder = LabelEncoder()
    train[col] = encoder.fit_transform(train[col])
    test[col] = encoder.transform(test[col])

In [ ]:
X = train.drop(['ECLO'], axis = 1)
y = train['ECLO']
X_test = test[X.columns]

In [ ]:
train.shape

(39609, 32)

In [ ]:
# skf = StratifiedKFold(n_splits = 9, shuffle = True, random_state = 42)
# folds = []
# models = {}
# for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
#   print('='*10)
#   print(f'Fold {fold}')
#   print('='*10)
#   X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
#   X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
#   model = CatBoostRegressor(random_state = 42) # tree_method = 'gpu_hist', gpu_id = 0
#   model.fit(X_train, y_train, verbose = 0)

#   y_pred = model.predict(X_val)
#   y_pred = np.round(y_pred, 0).astype(int)
#   #rmsle_scorer = rmsle(y_val, y_pred)
#   rmse_scorer = rmse(y_val, y_pred)
#   #print(f'RMSLE: {rmsle_scorer}')
#   print(f'RMSE: {rmse_scorer}')
#   models[fold] = model

# for fold in range(9):
#   sample_submission['ECLO'] += models[fold].predict(X_test) / 9

# sample_submission.loc[sample_submission['ECLO'] < 0.0, 'ECLO'] = 0.0

In [ ]:
# sample_submission.to_csv('./catboost.csv', index = False)

automl

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
# kfold = KFold(n_splits = 3, shuffle = True, random_state = 42)

# preds = 0
# for fold, (train_idx, val_idx) in enumerate(kfold.split(X)):
#   print(fold)
#   model = AutoML(mode="Compete",
#               algorithms = ['LightGBM', 'Xgboost', 'CatBoost'],
#               n_jobs = -1, total_time_limit = 3600 * 5, eval_metric="rmse", ml_task = "regression",)
#   trainX, trainy = X.iloc[train_idx], y.iloc[train_idx]
#   valX, valy = X.iloc[val_idx], y.iloc[val_idx]
#   model.fit(trainX, trainy)
#   sample_submission['ECLO'] += model.predict(X_test) / 3

In [ ]:
#fold_preds = np.sum(preds, 0) / 5

In [ ]:
automl = AutoML(mode="Compete",
                algorithms = ['Random Forest', 'LightGBM', 'Xgboost', 'CatBoost'],
                n_jobs = -1,total_time_limit=3600*5, eval_metric="rmse", ml_task = "regression",)
automl.fit(X, y)

AutoML directory: AutoML_5
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Random Forest', 'LightGBM', 'Xgboost', 'CatBoost']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree rmse 3.221812 trained in 0.95 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 4 models
1_Default_LightGBM rmse 3.243308 trained in 18.99 seconds
There was an error during 2_Default_Xgboost training.
Please check AutoML_5/errors.md for details.
There was an error during 3_De

ImportError: ignored

In [ ]:
pred = automl.predict(X_test)
sample_submission['ECLO'] = pred
sample_submission.loc[sample_submission['ECLO'] < 0.0, 'ECLO'] = 0.0
sample_submission.to_csv('./automl_fillna.csv', index = False)

In [ ]:
sample_submission

autogluon

In [ ]:
train_data = TabularDataset(train)
test_data = TabularDataset(test)

predictor = TabularPredictor(label='ECLO', eval_metric='rmse').fit(train_data, presets='high_quality') # , ag_args_fit={'num_gpus': 1}
# predictor = TabularPredictor(label='ECLO',  eval_metric='rmse').fit(train_data, time_limit=43200, presets='high_quality',  ag_args_fit={'num_gpus': 1},
#                                                                                      hyperparameters={'RF':{}, 'GBM':{},'XGB':{},'XT':{},'CAT':{}})

pred = predictor.predict(test_data)
sample_submission['ECLO'] = pred
sample_submission.loc[sample_submission['ECLO'] < 0.0, 'ECLO'] = 0.0
sample_submission.to_csv('./autogluon.csv', index = False)

No path specified. Models will be saved in: "AutogluonModels/ag-20231210_143937"
Presets specified: ['high_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 3600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20231210_143937/ds_sub_fit/sub_fit_ho.
Beginning AutoGluon training ... Time limit = 900s
AutoGluon will save models to "AutogluonModels/ag-20231210_143937/ds_sub_fit/sub_fit_ho"
=================== System Info ===================
AutoGluon Version:  1.

In [ ]:
sample_submission

,ID,ECLO
0,ACCIDENT_39609,3
1,ACCIDENT_39610,3
2,ACCIDENT_39611,3
3,ACCIDENT_39612,3
4,ACCIDENT_39613,3
...,...,...
10958,ACCIDENT_50567,3
10959,ACCIDENT_50568,3
10960,ACCIDENT_50569,3
10961,ACCIDENT_50570,3


deppfm

In [ ]:
!pip install --upgrade deepctr-torch

  Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.1.0
    Uninstalling urllib3-2.1.0:
      Successfully uninstalled urllib3-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
yfinance 0.2.32 requires requests>=2.31, but you have requests 2.28.2 which is incompatible.


In [ ]:
pip install --upgrade urllib3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
botocore 1.33.11 requires urllib3<2.1,>=1.25.4; python_version >= "3.10", but you have urllib3 2.1.0 which is incompatible.
requests 2.28.2 requires urllib3<1.27,>=1.21.1, but you have urllib3 2.1.0 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
yfinance 0.2.32 requires requests>=2.31, but you have requests 2.28.2 which is incompatible.


In [ ]:
import torch
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import *

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# 요일, 기상상태, 시군구, 도로형태, 노면상태, 사고유형, 구, 동, 사고요일, 계절, 사고주말여부, 시간대, 도로노선방향, 단속구분, 제한속도, 보호구역도로폭
# CCTV개수, 설치개수, CCTV설치대수, 초등학교개수, 주차장개수, 급지_1, 급지_2, 급지_3, 주차장유형_노상, 주차장유형_노외, 사고발생횟수

# 범주형 특성
sparse_features = ['요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', '구', '동', '사고요일', '계절', '사고주말여부', '시간대', '도로노선방향', '단속구분', '제한속도', '보호구역도로폭']

# 숫자형 특성
dense_features = ['CCTV개수', '설치개수', 'CCTV설치대수', '설치형태_전용주', '설치형태_한전주', '초등학교개수', '주차장개수', '급지_1', '급지_2', '급지_3', '주차장유형_노상', '주차장유형_노외', '사고발생횟수']

train[sparse_features] = train[sparse_features].fillna(-1)
train[dense_features] = train[dense_features].fillna(0)
test[sparse_features] = test[sparse_features].fillna(-1)
test[dense_features] = test[dense_features].fillna(0)
target = 'ECLO'

# for feat in sparse_features:
#   encoder = LabelEncoder()
#   train[feat] = encoder.fit_transform(train[feat])
#   test[feat] = encoder.transform(test[feat])

scaler = MinMaxScaler(feature_range = (0, 1))
train[dense_features] = scaler.fit_transform(train[dense_features])
test[dense_features] = scaler.fit_transform(test[dense_features])

In [ ]:
fixlen_feature_columns = [SparseFeat(feat, train[feat].nunique()) for feat in sparse_features] + [DenseFeat(feat, 1, ) for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [ ]:
X_train, X_val = train_test_split(train, test_size = 0.2, random_state = 42)

train_model_input = {name: train[name] for name in feature_names}
val_model_input = {name: X_val[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}

In [ ]:
device = 'cpu'
use_cuda = True

if use_cuda and torch.cuda.is_available():
    print('cuda')
    device = 'cuda:0'

batch_size = 256
epoch = 10
verbose = 2

model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns, task='regression', device=device)

model.compile(optimizer='adam', loss='mse', metrics=['mse'])

model.fit(train_model_input, train[target].values, batch_size = batch_size, epoch = epoch, verbose = verbose, vaildation_split = 0.2)
val_pred = model.predict(val_model_input, 256)
print('Test MSE: ', round(mean_squared_error(X_val[target].values, val_pred), 4))

TypeError: ignored